In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
dir_processed = "../data/processed"

In [ ]:
matches = pd.read_csv(dir_processed+"matches.csv", index_col=0)

In [ ]:
matches.head()

In [ ]:
def calculate_expected_score(
    rating1: float,
    rating2: float
) -> float:
    """
    Calculate the expected score using ELO rating system
    
    Parameters
    ----------
    rating1, rating2: float
        ELO ratings of two players
    
    Returns
    -------
    float
        The expected score of a player with rating1 against
        a player with rating2 using the ELO rating system
    """
    return 1 / (1 + 10**((rating2 - rating1) / 400))

# list the expected scores for various differences in ratings
for delta in range(0, 501, 20):
    print(f'Difference in rating: {delta:3}.' +
          f'Expected score: {calculate_expected_score(delta, 0)}')